## Microsoft Entra ID の概要

Microsoft Entra ID（旧 Azure Active Directory）は、Microsoft のクラウドベースの ID およびアクセス管理サービスです。Microsoft 365、Azure、およびその他数千の SaaS アプリケーションの中央 ID プロバイダーとして機能します。

主な機能:
* **シングルサインオン（SSO）** - ユーザーは一度認証するだけで複数のアプリケーションにアクセス可能
* **多要素認証（MFA）** - 追加の検証方法によるセキュリティ強化
* **条件付きアクセス** - ユーザー、デバイス、場所、リスクに基づくポリシーベースのアクセス制御
* **アプリケーション統合** - OAuth 2.0、OpenID Connect、SAML などの最新の認証プロトコルをサポート

## Amazon Bedrock Gateway の概要

Bedrock AgentCore Gateway は、既存の API や Lambda 関数をインフラやホスティングを管理することなく、フルマネージドの MCP サーバーに変換する方法を提供します。既存の API に対して OpenAPI 仕様や Smithy モデルを持ち込むことも、ツールのフロントエンドとして Lambda 関数を追加することもできます。Gateway はこれらすべてのツールに対して統一された Model Context Protocol（MCP）インターフェースを提供します。Gateway は、受信リクエストとターゲットリソースへの送信接続の両方に対して安全なアクセス制御を確保するために、デュアル認証モデルを採用しています。このフレームワークは 2 つの主要コンポーネントで構成されています：Inbound Auth は Gateway ターゲットにアクセスしようとするユーザーを検証・認可し、Outbound Auth は認証されたユーザーに代わって Gateway がバックエンドリソースに安全に接続できるようにします。これらの認証メカニズムにより、IAM 認証情報と OAuth ベースの認証フローの両方をサポートする、ユーザーとターゲットリソース間の安全なブリッジが作成されます。Gateway は MCP の Streamable HTTP トランスポート接続をサポートしています。

Amazon Bedrock AgentCore Gateway の詳細については以下を参照してください：
- https://github.com/awslabs/amazon-bedrock-agentcore-samples/tree/main/01-tutorials/02-AgentCore-gateway
- https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/gateway.html

## 学習目標
Microsoft EntraID は、AgentCore Identity の ID プロバイダーとして使用でき、保護された Amazon AgentCore Gateway リソースへのアプリケーションのアクセスを認可できます。このノートブックでは、Amazon Bedrock Gateway での Inbound 認証に EntraID を使用する方法を学びます。

## 学習目標 1: AgentCore Gateway で使用するための Entra ID のセットアップ

### ステップ 1: Entra ID テナントのセットアップ
Entra ID テナントは、組織を表す Microsoft Entra ID の専用インスタンスです。Microsoft のクラウド内にある組織の独立したディレクトリと考えてください。

主な特徴:
* **一意の ID** - 各テナントは一意のドメインを持ちます（例：yourcompany.onmicrosoft.com）
* **分離された境界** - あるテナント内のユーザー、グループ、アプリケーションは他のテナントから分離されています
* **管理制御** - テナント管理者がユーザー、セキュリティポリシー、アプリケーション登録を管理します
* **マルチドメインサポート** - デフォルトの .onmicrosoft.com ドメインとカスタムドメインの両方を含めることができます

実際の運用:
OAuth 統合のために Entra ID にアプリケーションを登録する場合、特定のテナント内に登録することになります。そのテナントのユーザーは、組織の資格情報を使用してアプリケーションに認証できます。

AgentCore 統合に必要なもの:
* **テナント ID** - Entra ID インスタンスの一意の識別子
* **アプリケーション登録** - テナント内に登録されたアプリ
* **適切な権限** - アプリケーションに設定されたアクセス権

このテナントベースのモデルにより、認証と認可が組織のセキュリティ境界内に維持されます。

テナントの作成手順については https://learn.microsoft.com/en-us/entra/fundamentals/create-new-tenant を参照してください

注意:
1. MS EntraID は AWS のサービスではありません。EntraID に関連するコストについては、Microsoft EntraID のドキュメントを参照してください。
2. 以下のステップで使用されているスクリーンショットは変更される可能性があります。EntraID アプリケーションのセットアップに関する最新のガイダンスについては、Microsoft Entra ID のドキュメントを参照することをお勧めします。

In [ ]:
import os
os.environ["tenant_id"] = "bcXXXXXX-CCCC-VVVV-BBBB-NNNNNNdf1f19" # Replace with Tenant ID from EntraID

### ステップ 2: 使用する API を定義する
1. portal.azure.com にアクセスし、画面上部の検索バーで「Entra ID」を検索します
<img src="images/entraid.jpg" width="75%">
2. 管理 --> アプリの登録 に移動します
<img src="images/app.registration.png" width="75%">
3. 「新規登録」をクリックし、詳細を入力します。マルチテナントオプションを選択してください
- リダイレクト URL は設定しないでください。
<img src="images/setup.api.png" width="75%">
4. 管理 --> API の公開 を通じて API を公開します
<img src="images/api.expose.png" width="75%"/>
5. API のアプリロールを作成します。これは M2M セットアップのため、スコープは追加しません。
<img src="images/weather.app.role.png" width="75%"/>

In [ ]:
os.environ["app_id_uri"] = "api://3dXXXXXX-CCCC-VVVV-BBBB-NNNNNN885f25" # This is the API URL you set up for "weather_service"

### ステップ 3: Entra クライアントアプリケーションを作成する
1. portal.azure.com にアクセスし、画面上部の検索バーで「Entra ID」を検索します
<img src="images/entraid.jpg" width="75%">
2. 管理 --> アプリの登録 に移動します
<img src="images/app.registration.png" width="75%">
3. 「新規登録」をクリックし、詳細を入力します。マルチテナントオプションを選択してください
- リダイレクト URL は設定しないでください。
<img src="images/client.register.png" width="75%"/>
4. クライアントシークレットを作成します。AgentCore で使用するためにクライアントシークレットとクライアント ID をコピーします。
<img src="images/client.secret.png" width="75%">
5. API のアクセス許可に移動し、先ほど作成した API のアクセス許可を要求します。
<img src="images/api.permissions.png" width="75%">
<img src="images/api.permissions.png" width="75%">
5. API を使用するための管理者の同意を付与します。
<img src="images/api.permissions.png" width="75%">

5. EntraID の情報を使用して環境変数を設定します

In [ ]:
import os
os.environ["client_id"] = "08XXXXXX-CCCC-VVVV-BBBB-NNNNNNd86cd2" # Replace with Client ID of the "weather_service_client"
os.environ["client_secret"] = "muCCCCCVVVVVBBBBBNNNNN3dY6qdlL" # Replace with Client secret of the "weather_service_client"

## 学習目標 2: AgentCore Gateway と Lambda ターゲットのセットアップ

### ステップ 1: Entra ID で使用する Lambda 関数を作成する
1. Lambda 関数のコードとして使用する Python ファイルを作成します。呼び出されるツール名が `context` オブジェクトから取得され、Lambda 関数で使用される点に注目してください。

In [ ]:
import boto3
import zipfile
import io
from botocore.exceptions import ClientError
from boto3.session import Session
import time

boto_session = Session()
sts = boto3.client('sts')
region = boto_session.region_name
account_id = sts.get_caller_identity().get("Account")

In [ ]:
%%writefile lambda_function.py
def lambda_handler(event, context):
    print(f"Event: {event}")
    print(f"Context: {context}")
    extended_tool_name = context.client_context.custom["bedrockAgentCoreToolName"]
    resource = extended_tool_name.split("___")[1]

    print(resource)
    city = event.get("city")
    print(city)
    if resource == "weather_check":
        return f"Weather in {city} is bright and sunny!"
    elif resource == "directions":
        return f"Take I5 south all the way to {city} downtown"

2. Lambda 関数を作成します

In [ ]:
lambda_client = boto3.client('lambda', region_name=region)
with zipfile.ZipFile('lambda_function.zip', 'w') as zip_file:
    zip_file.write('lambda_function.py', 'lambda_function.py')

with open('lambda_function.zip', 'rb') as zip_file:
    zip_content = zip_file.read()

In [ ]:
iam_client = boto3.client('iam', region_name=region)

trust_policy = """{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
"""

policy = """{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "logs:CreateLogGroup",
                "logs:CreateLogStream",
                "logs:PutLogEvents"
            ],
            "Resource": "arn:aws:logs:*:*:*"
        }
    ]
}
"""

response = iam_client.create_role(
    RoleName='lambda-role',
    AssumeRolePolicyDocument=trust_policy
)

iam_client.put_role_policy(
        PolicyDocument=policy,
        PolicyName="lambda-policy",
        RoleName="lambda-role"
    )

lambda_role_arn = response['Role']['Arn']

# Wait for role to propagate
time.sleep(10)

response = lambda_client.create_function(
    FunctionName='m2m-entra-lambda',
    Runtime='python3.12',
    Role=lambda_role_arn,
    Handler='lambda_function.lambda_handler',
    Code={'ZipFile': zip_content},
)

In [ ]:
lambda_arn = response["FunctionArn"]

In [ ]:
lambda_arn

### ステップ 2: Inbound セキュリティを備えた Amazon Bedrock AgentCore Gateway を作成する

In [ ]:
iam_client = boto3.client('iam')

trust_policy = """{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock-agentcore.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
"""

# Create role with trust policy
response = iam_client.create_role(
    RoleName='bedrock-agent-lambda-role',
    AssumeRolePolicyDocument=trust_policy
)

permission = """{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "lambda:InvokeFunction"
            ],
            "Resource": [
                "%s"
            ],
            "Effect": "Allow",
            "Sid": "InvokeFunction"
        }
    ]
}
"""% lambda_arn


# Add Lambda invoke policy
iam_client.put_role_policy(
    RoleName='bedrock-agent-lambda-role',
    PolicyName='lambda-invoke-policy',
    PolicyDocument=permission
)

role_arn = response['Role']['Arn']
print(f"Role ARN: {role_arn}")

In [ ]:
gateway_client = boto3.client(
    "bedrock-agentcore-control",
    region_name=region,
)

gateway_name = "m2m-entra-gateway"
auth_config = {
    "customJWTAuthorizer": {
        "allowedAudience": [
            os.environ["app_id_uri"]
        ],
        "discoveryUrl": f"https://login.microsoftonline.com/{os.environ["tenant_id"]}/.well-known/openid-configuration"
    }
}

create_response = gateway_client.create_gateway(
    name=gateway_name,
    roleArn= role_arn,
    protocolType="MCP",
    authorizerType="CUSTOM_JWT",
    authorizerConfiguration=auth_config,
    description="Customer Support AgentCore Gateway",
)

In [ ]:
gateway_url = create_response["gatewayUrl"]
gateway_id = create_response["gatewayId"]

### ステップ 3: 作成した AgentCore Gateway に Lambda ターゲットを追加する

1. Lambda 関数を通じて作成する実際のツールの API 仕様です。

In [ ]:
api_spec = [
    {
        "name": "weather_check",
        "description": "Check the weather for a given City",
        "inputSchema": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city you want to get weather for"
                }
            },
            "required": [
                "city"
            ]
        }
    },
    {
        "name": "directions",
        "description": "Search the web for directions to a city",
        "inputSchema": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city you want to get directions to"
                }
            },
            "required": [
                "city"
            ]
        }
    }
]

In [ ]:
lambda_target_config = {
    "mcp": {
        "lambda": {
            "lambdaArn": lambda_arn,
            "toolSchema": {"inlinePayload": api_spec},
        }
    }
}

# Create gateway target
credential_config = [{"credentialProviderType": "GATEWAY_IAM_ROLE"}]

create_target_response = gateway_client.create_gateway_target(
    gatewayIdentifier=gateway_id,
    name="LambdaUsingSDK",
    description="Lambda Target using SDK",
    targetConfiguration=lambda_target_config,
    credentialProviderConfigurations=credential_config,
)

## 学習目標 3: AgentCore Gateway を通じて利用可能になったツールをエージェントで使用する

### ステップ 1: トークンを取得し、ペイロードとヘッダーを確認する
1. アクセストークンを取得し、それを使用して AgentCore Gateway にアクセスします。

In [ ]:
import requests
import json

TOKEN_URL = f"https://login.microsoftonline.com/{os.environ["tenant_id"]}/oauth2/v2.0/token"
SCOPE=f"{os.environ["app_id_uri"]}/.default"

def fetch_access_token(client_id, client_secret, token_url,scope):

  data = {
      "grant_type":"client_credentials",
      "client_id":client_id,
      "client_secret": client_secret,
      "scope":scope
  }
    
  response = requests.post(
    token_url,
    data=data,
    headers={'Content-Type': 'application/x-www-form-urlencoded'}
  )
  #print(response.text)
  return response.json()['access_token']

access_token = fetch_access_token(os.environ["client_id"], os.environ["client_secret"], TOKEN_URL, SCOPE)

2. デコードして内容を確認します。「aud」、「appid」、「roles」が先ほどセットアップしたものと一致していることを確認してください。

In [ ]:
import base64
import json

def decode_jwt_token(token):
    # Split the JWT into parts
    parts = token.split('.')
    
    # Decode header
    header = json.loads(base64.b64decode(parts[0] + '==').decode('utf-8'))
    
    # Decode payload
    payload = json.loads(base64.b64decode(parts[1] + '==').decode('utf-8'))
    
    return header, payload

# Usage
header, payload = decode_jwt_token(access_token)

print("Header:", json.dumps(header, indent=2))
print("Payload:", json.dumps(payload, indent=2))

# Check specific claims
print(f"Audience: {payload.get('aud')}")
print(f"Issuer: {payload.get('iss')}")
print(f"Expires: {payload.get('exp')}")
print(f"Scopes: {payload.get('scp')}")
print(f"Roles: {payload.get('roles')}")

### ステップ 2: アクセストークンを使用して AgentCore Gateway から利用可能なツールのリストを取得する
以下のようなツール仕様が表示されるはずです。

<img src="images/tools.spec.png" width="50%"/>

In [ ]:
def list_tools(gateway_url, access_token):
  headers = {
      "Content-Type": "application/json",
      "Authorization": f"Bearer {access_token}"
  }

  payload = {
      "jsonrpc": "2.0",
      "id": "list-tools-request",
      "method": "tools/list"
  }

  response = requests.post(gateway_url, headers=headers, json=payload)
  return response.json()
tools = list_tools(gateway_url, access_token)
print(json.dumps(tools, indent=2))

### ステップ 3: MCP クライアントを作成し、ツールリストを取得して Strands エージェントで使用する

In [ ]:
from mcp.client.streamable_http import streamablehttp_client
from strands.tools.mcp import MCPClient

# Set up MCP client
mcp_client = MCPClient(
    lambda: streamablehttp_client(
        gateway_url,
        headers={"Authorization": f"Bearer {access_token}"},
    )
)

In [ ]:
mcp_client.start()

In [ ]:
mcp_client.list_tools_sync()

In [ ]:
from strands import Agent
agent = Agent(tools=mcp_client.list_tools_sync())

#### 注意: 先ほど定義した Lambda 関数からのレスポンスは静的です。そのため、プロンプトでどの都市を指定しても、このエージェントからのレスポンスは非常に似たものになります。

In [ ]:
agent("What is the weather in San Diego?")

In [ ]:
agent("Give me directions to San Diego?")

## まとめとクリーンアップ
このノートブックでは以下のことを学びました：
- OAuth クライアント認証情報（M2M）フローを提供するための Entra ID API とアプリケーションのセットアップ
- AgentCore Gateway の作成
- Lambda 関数を作成し、作成した AgentCore Gateway のターゲットとして追加。Lambda 関数は AgentCore Gateway を通じて MCP ツールとして利用可能になります。
- MCP クライアントを使用して Gateway を通じて提供されるツールにアクセスし、ツールを Strands Agent にバインドして、ユーザーのクエリに対応する

#### 作成されたリソース

In [ ]:
lambda_arn, role_arn, gateway_id, lambda_role_arn, create_response["gatewayArn"]

In [ ]:
create_target_response["targetId"]

#### Gateway の Lambda ターゲットを削除

In [ ]:
gateway_client.delete_gateway_target(gatewayIdentifier=gateway_id, targetId=create_target_response["targetId"])

#### Gateway を削除

In [ ]:
gateway_client.delete_gateway(gatewayIdentifier=gateway_id)

#### 作成した Lambda 関数を削除

In [ ]:
function_name = lambda_arn.split(':')[-1]
lambda_client.delete_function(FunctionName=function_name)

#### 作成したロールを削除

In [ ]:
role_name = lambda_role_arn.split('/')[-1]
inline = iam_client.list_role_policies(RoleName=role_name)
for policy_name in inline['PolicyNames']:
    iam_client.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
iam_client.delete_role(RoleName=role_name)

In [ ]:
role_name = role_arn.split('/')[-1]
inline = iam_client.list_role_policies(RoleName=role_name)
for policy_name in inline['PolicyNames']:
    iam_client.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
iam_client.delete_role(RoleName=role_name)